# Silver Layer Transformation - Race Control Messages

## Objective
Transform the `bronze.race_control_messages` table into a clean and structured Silver table.

## Steps
1. **Read** data from Bronze table `bronze.race_control_messages`.
2. **Select & Rename** important columns:
   - `Scope`, `Category`, `Flag`, `Message`, `Status`
   - `Date` (cast to `timestamp`)
   - `session_key`
3. **Data Cleaning**
   - Standardize string columns (trim spaces, uppercase flags).
   - Cast `Date` into a proper timestamp.
   - Drop duplicates.
4. **Write** results into Silver schema as `silver.race_control_messages`.

## Output
- Clean and structured Silver table:
  `silver.race_control_messages`


In [0]:
from pyspark.sql import functions as F

# Read from Bronze
rcm_bronze = spark.table("bronze.race_control_messages")

# Transformations
rcm_silver = (
    rcm_bronze
    .select(
        F.col("Scope").alias("scope"),
        F.col("Category").alias("category"),
        F.upper(F.col("Flag")).alias("flag"),
        F.col("Message").alias("message"),
        F.col("Status").alias("status"),
        F.col("session_key").cast("string").alias("session_key"),
        F.to_timestamp("Date").alias("message_timestamp")
    )
    .dropDuplicates()
)

# Write to Silver
(
    rcm_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver.race_control_messages")
)